<a href="https://colab.research.google.com/github/cchen744/uhi-extreme-heat-response/blob/main/notebooks/03_uhi_n_landcover.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ΔUHI & Built-environment
1. **Goal**: To investigate the possible relationship between SUHI response and built environment. Since we observed a significant increase in SUHI in several cities while others not, we are assuming that if this condition-dependence might be contributed to by composition of the built environment; we test whether this is true.

2. **Outcome Variable: Defining SUHI condition-dependence**: in this step, we define SUHI's condition-dependence as:
    
    *ΔUHI = UHI_extreme − UHI_baseline*

    - UHI_extreme: SUHI when the daily mean temperature is over 90 percentile of the daily mean temperature
    - UHI_baseline: the average of SUHI when the daily mean temperature is between 50-70 percentile.

    (Percentiles are computed within the warm-season window to ensure comparability across cities.)

3. **Explanatory Variables: Built-environment characteristics**:

    **Surface composition**:
      - Impervious surface fraction
      - Vegetation / NDVI / tree cover
      - Water or bare land fraction
      - LCZ composition

    **Urban form & intensity proxies**:
      - Built-up density / road density

4. **Analytical Strategy**
  - Analytical Unit: grid cell within each city (resolution =
  - Model:
  
    *ΔUHI_cell ~ composition_cell + proxies_cell + city fixed effects*

  - Comparison logic:
    - within-city: which built factors is correlated with ΔUHI_cell
    - across-city: does this explain why some city has higher ΔUHI
  
  - Control principles:
    - Same buffer scale
    - Same spatial resolution
    - Same seasonal window



In [1]:
!git init
!git remote add origin https://github.com/cchen744/uhi-extreme-heat-response.git
!git pull origin main --allow-unrelated-histories
!git branch -m master
!git status

hint: Using 'master' as the name for the initial branch. This default branch name
hint: is subject to change. To configure the initial branch name to use in all
hint: of your new repositories, which will suppress this warning, call:
hint: 
hint: 	git config --global init.defaultBranch <name>
hint: 
hint: Names commonly chosen instead of 'master' are 'main', 'trunk' and
hint: 'development'. The just-created branch can be renamed via this command:
hint: 
hint: 	git branch -m <name>
Initialized empty Git repository in /content/.git/
remote: Enumerating objects: 248, done.
remote: Counting objects: 100% (89/89), done.
remote: Compressing objects: 100% (69/69), done.
remote: Total 248 (delta 41), reused 41 (delta 15), pack-reused 159 (from 2)
Receiving objects: 100% (248/248), 10.23 MiB | 13.04 MiB/s, done.
Resolving deltas: 100% (102/102), done.
From https://github.com/cchen744/uhi-extreme-heat-response
 * branch            main       -> FETCH_HEAD
 * [new branch]      main       -> origin

In [2]:
from pathlib import Path
import os
import pandas as pd
import ee
import uhi_pipeline
import importlib
importlib.reload(uhi_pipeline)
print("uhi_pipeline module reloaded.")

ee.Authenticate()
ee.Initialize(project='extremeweatheruhi')

DATA_DIR = Path("data/city_cell")
DATA_DIR.mkdir(parents=True, exist_ok=True)

ua_fc = ee.FeatureCollection("projects/extremeweatheruhi/assets/uac20_2025")

uhi_pipeline module reloaded.


Before starting analysis, we need to get daily output on the level of grid cell due to the need for intra-city built environment analysis. In uhi_pipeline.py, the grid cell is defined as a 1km x 1km square grid projected from EPSG: 3875.

In [3]:
city_fc=uhi_pipeline.select_ua(ua_fc,ua_contains="Phoenix")
city_geom = city_fc.geometry()
ic = (ee.ImageCollection("MODIS/061/MYD11A1")
      .filterBounds(city_geom)
      .filterDate("2013-07-01", "2013-08-01"))
print("IC count:", ic.size().getInfo())

IC count: 31


In [24]:
lcz_img = ee.ImageCollection("RUB/RUBCLIM/LCZ/global_lcz_map/latest").first()
lcz = lcz_img.select("LCZ_Filter")
lst_scale_m = 1000
ring_outer_m= 12000
ring_inner_m= 3000
agg_func="median"
cell_crs="EPSG:3857"

start_date="2013-07-01"
end_date="2013-08-01"
unit="cell", # modify unit to 'cell'
cell_scale_m=500
lst_band="LST_Night_1km"
qc_band="QC_Night"

BUILT_MIN, BUILT_MAX = 1, 10
WATER_CODE = 17

is_built = lcz.gte(BUILT_MIN).And(lcz.lte(BUILT_MAX))
is_water = lcz.eq(WATER_CODE)
is_natural = is_built.Not().And(is_water.Not())

urban_region = city_geom
outer = city_geom.buffer(ring_outer_m)
inner = city_geom.buffer(ring_inner_m)
rural_region = outer.difference(inner)

urban_mask = is_water.Not().clip(urban_region)
rural_mask = is_natural.clip(rural_region)

In [10]:
grid_fc = uhi_pipeline.make_grid_fc_2(urban_region, cell_size_m=lst_scale_m, crs="EPSG:3857")
print("grid size:", grid_fc.size().getInfo())

grid size: 1


In [11]:
img = ic.first()
urb = img.updateMask(urban_mask)
urb_cells = urb.reduceRegions(collection=grid_fc, reducer=ee.Reducer.count(), scale=lst_scale_m)
print("urb_cells size:", urb_cells.size().getInfo())

urb_cells size: 1


In [15]:
test_img = ic.first()
urban_count = test_img.updateMask(urban_mask).reduceRegion(
    reducer=ee.Reducer.count(),
    geometry=urban_region,
    scale=lst_scale_m,
    maxPixels=1e13
)
rural_count = test_img.updateMask(rural_mask).reduceRegion(
    reducer=ee.Reducer.count(),
    geometry=rural_region,
    scale=lst_scale_m,
    maxPixels=1e13
)

print("urban count:", urban_count.getInfo())
print("rural count:", rural_count.getInfo())

urban count: {'Clear_day_cov': 3207, 'Clear_night_cov': 580, 'Day_view_angle': 3207, 'Day_view_time': 3207, 'Emis_31': 3208, 'Emis_32': 3208, 'LST_Day_1km': 3207, 'LST_Night_1km': 580, 'Night_view_angle': 580, 'Night_view_time': 580, 'QC_Day': 3211, 'QC_Night': 3210}
rural count: {'Clear_day_cov': 3435, 'Clear_night_cov': 1413, 'Day_view_angle': 3435, 'Day_view_time': 3435, 'Emis_31': 3435, 'Emis_32': 3435, 'LST_Day_1km': 3435, 'LST_Night_1km': 1413, 'Night_view_angle': 1413, 'Night_view_time': 1413, 'QC_Day': 3436, 'QC_Night': 3411}


In [20]:
importlib.reload(uhi_pipeline)
img = ic.first()
urb = img.updateMask(urban_mask)
stats = urb.reduceRegion(
    reducer=ee.Reducer.mean().combine(ee.Reducer.count(), sharedInputs=True),
    geometry=urban_region,
    scale=lst_scale_m,
    maxPixels=1e13
)
print(stats.getInfo())

{'Clear_day_cov_count': 3876, 'Clear_day_cov_mean': 2609.521054813561, 'Clear_night_cov_count': 774, 'Clear_night_cov_mean': 1570.446962489805, 'Day_view_angle_count': 3876, 'Day_view_angle_mean': 20.97918549490695, 'Day_view_time_count': 3876, 'Day_view_time_mean': 128, 'Emis_31_count': 3877, 'Emis_31_mean': 240.8071150700404, 'Emis_32_count': 3877, 'Emis_32_mean': 243.28913484132346, 'LST_Day_1km_count': 3876, 'LST_Day_1km_mean': 16349.599981166597, 'LST_Night_1km_count': 774, 'LST_Night_1km_mean': 14962.769033704799, 'Night_view_angle_count': 774, 'Night_view_angle_mean': 44.303438434357155, 'Night_view_time_count': 774, 'Night_view_time_mean': 17.88910029899428, 'QC_Day_count': 3880, 'QC_Day_mean': 59.25756211838235, 'QC_Night_count': 3879, 'QC_Night_mean': 13.40320235919588}


In [28]:
fc_raw = uhi_pipeline.make_daily_table_cells(
    start_date, end_date,
    urban_region, rural_region,
    urban_mask, rural_mask,
    lst_band, qc_band,
    agg_func,
    lst_scale_m=lst_scale_m,
    cell_scale_m=cell_scale_m,
    crs=cell_crs
)

print("raw fc size:", fc_raw.size().getInfo())
print("raw first feature:", fc_raw.first().getInfo())

raw fc size: 31
raw first feature: {'type': 'Feature', 'geometry': {'type': 'MultiPolygon', 'coordinates': [[[[-111.64856855917273, 33.28237353808885], [-111.64855518588121, 33.280687986070916], [-111.64853735483487, 33.278913273697114], [-111.64853289707106, 33.278021421117565], [-111.64853289707106, 33.27787426174826], [-111.6467091263499, 33.27786088844114], [-111.64435915760556, 33.27783862996473], [-111.64356986066059, 33.27783416268381], [-111.64356986066059, 33.278008097129145], [-111.64357877618825, 33.2788687145058], [-111.64359214947976, 33.28064338247106], [-111.64360998052611, 33.282427082969924], [-111.64845702414043, 33.282467219147684], [-111.64848377071445, 33.2824582682677], [-111.6485150660247, 33.28244489805095], [-111.64855072812638, 33.28241367505721], [-111.64856855917273, 33.28237353808885]]], [[[-112.3232617229914, 33.39181773725829], [-112.32324835261946, 33.391532317618754], [-112.32323052546279, 33.391081944778676], [-112.32314129871617, 33.3883217824071], [-

In [29]:
print("raw props keys:", fc_raw.first().propertyNames().getInfo())
print("raw props values:", fc_raw.first().toDictionary().getInfo())

raw props keys: ['system:index', 'date', 'LST_rur', 'rural_n', 'cell_id']
raw props values: {'LST_rur': 26.784184239733648, 'cell_id': '-12472000_3962644', 'date': '2013-07-01', 'rural_n': 1609}


In [30]:
img = ic.first()
urb = img.updateMask(urban_mask)

#
cell = grid_fc.first().geometry()

#
urban_count = urb.reduceRegion(
    reducer=ee.Reducer.count(),
    geometry=cell,
    scale=lst_scale_m,
    maxPixels=1e13
)
print("urban_count in cell:", urban_count.getInfo())

urban_count in cell: {'Clear_day_cov': 3207, 'Clear_night_cov': 580, 'Day_view_angle': 3207, 'Day_view_time': 3207, 'Emis_31': 3208, 'Emis_32': 3208, 'LST_Day_1km': 3207, 'LST_Night_1km': 580, 'Night_view_angle': 580, 'Night_view_time': 580, 'QC_Day': 3211, 'QC_Night': 3210}


In [12]:
# Example: Phoenix
importlib.reload(uhi_pipeline)
df = uhi_pipeline.run_city(
    ua_fc=ua_fc,
    ua_contains="Phoenix",
    start_date="2013-07-01",
    end_date="2013-08-01",
    unit="cell", # modify unit to 'cell'
    cell_scale_m=500,
    min_cell_pixels=0,
    min_rural_pixels=0,
    agg_func="median",
    lst_band="LST_Night_1km",
    qc_band="QC_Night",
)

print(df.shape)
print(df.head())
# print(df["date"].min(), df["date"].max())
# print(df[["cell_n","rural_n"]].describe())

fc_list length: 1
first fc size: 0
No features returned after filtering. Retrying with relaxed pixel thresholds: urban>=1, rural>=1, cell>=1.
No features returned from Earth Engine after filtering. Returning empty DataFrame.
(0, 0)
Empty DataFrame
Columns: []
Index: []


Displaying the current content of `uhi_pipeline.py` to verify the applied changes.

In [8]:
with open('/content/uhi_pipeline.py', 'r') as f:
    print(f.read())

"""
Reusable SUHI data pipeline (GEE + MODIS LST)
Optimized Version: Uses Combined Reducers to reduce server load.
"""

import ee
import geemap
import pandas as pd
import numpy as np
from datetime import datetime
from dateutil.relativedelta import relativedelta

# ------------------------------------------------------------------
# 0. Earth Engine init
# ------------------------------------------------------------------
def init_ee():
    try:
        ee.Initialize()
    except Exception:
        ee.Authenticate()
        ee.Initialize()

# ------------------------------------------------------------------
# 1. Helper: monthly ranges
# ------------------------------------------------------------------
def month_starts(start_date, end_date):
    s = datetime.strptime(start_date, "%Y-%m-%d").replace(day=1)
    e = datetime.strptime(end_date, "%Y-%m-%d")
    cur = s
    while cur < e:
        nxt = cur + relativedelta(months=1)
        yield cur.strftime("%Y-%m-%d"), nxt.strftime("%Y-%m-%